<a href="https://www.kaggle.com/code/kumarkatariya/predicting-heart-disease-baseline-simple?scriptVersionId=295669921" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s6e2/train.csv',index_col='id')
train.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
id,,,,,,,,,,,,,,
0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [3]:
test = pd.read_csv('/kaggle/input/playground-series-s6e2/test.csv')
test.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,630000,58,1,3,120,288,0,2,145,1,0.8,2,3,3
1,630001,55,0,2,120,209,0,0,172,0,0.0,1,0,3
2,630002,54,1,4,120,268,0,0,150,1,0.0,2,3,7
3,630003,44,0,3,112,177,0,0,168,0,0.9,1,0,3
4,630004,43,1,1,138,267,0,0,163,0,1.8,2,0,7


In [4]:
test_ids = test['id']

In [5]:
test.drop('id',axis=1, inplace=True)

In [6]:
test.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,58,1,3,120,288,0,2,145,1,0.8,2,3,3
1,55,0,2,120,209,0,0,172,0,0.0,1,0,3
2,54,1,4,120,268,0,0,150,1,0.0,2,3,7
3,44,0,3,112,177,0,0,168,0,0.9,1,0,3
4,43,1,1,138,267,0,0,163,0,1.8,2,0,7


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 630000 entries, 0 to 629999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Age                      630000 non-null  int64  
 1   Sex                      630000 non-null  int64  
 2   Chest pain type          630000 non-null  int64  
 3   BP                       630000 non-null  int64  
 4   Cholesterol              630000 non-null  int64  
 5   FBS over 120             630000 non-null  int64  
 6   EKG results              630000 non-null  int64  
 7   Max HR                   630000 non-null  int64  
 8   Exercise angina          630000 non-null  int64  
 9   ST depression            630000 non-null  float64
 10  Slope of ST              630000 non-null  int64  
 11  Number of vessels fluro  630000 non-null  int64  
 12  Thallium                 630000 non-null  int64  
 13  Heart Disease            630000 non-null  object 
dtypes: float6

In [8]:
train.duplicated().sum()

np.int64(0)

In [9]:
target = train['Heart Disease'] 

In [10]:
train.drop('Heart Disease',inplace=True,axis=1)

In [11]:
train.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
id,,,,,,,,,,,,,
0,58,1,4,152,239,0,0,158,1,3.6,2,2,7
1,52,1,1,125,325,0,2,171,0,0.0,1,0,3
2,56,0,2,160,188,0,2,151,0,0.0,1,0,3
3,44,0,3,134,229,0,2,150,0,1.0,2,0,3
4,58,1,4,140,234,0,2,125,1,3.8,2,3,3


In [12]:
target = target.map({'Absence':0,'Presence':1})

In [13]:
X = train.copy()
y = target.copy()

In [14]:
from xgboost import XGBClassifier 
from sklearn.metrics import roc_auc_score 

In [15]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'n_estimators': 5000,
    'learning_rate': 0.02,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1
} 

In [16]:
from sklearn.model_selection import StratifiedKFold
N_Splits = 5 
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test)) 

for train_idx,val_idx in skf.split(X,y): 
    X_train,X_val = X.iloc[train_idx],X.iloc[val_idx]
    y_train,y_val = y.iloc[train_idx],y.iloc[val_idx]

    model = XGBClassifier(**xgb_params)
    model.fit(X_train,y_train) 

    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_preds 
    
    fold_auc = roc_auc_score(y_val, val_preds)
    print(f"Fold AUC: {fold_auc:.5f}") 
    
    test_preds+= model.predict_proba(test)[:,1]/N_Splits

cv_auc = roc_auc_score(y, oof_preds)
print(f"\nOverall CV AUC: {cv_auc:.5f}") 

pd.DataFrame({'oof_preds_xgb':oof_preds}).to_csv('oof_preds_xgb.csv',index=False)
pd.DataFrame({'id':test_ids,'test_preds':test_preds}).to_csv('test_preds.csv',index=False)

Fold AUC: 0.95531
Fold AUC: 0.95433
Fold AUC: 0.95517
Fold AUC: 0.95471
Fold AUC: 0.95558

Overall CV AUC: 0.95502


In [17]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s6e2/sample_submission.csv')

In [18]:
sample_submission.head()

,id,Heart Disease
0,630000,0
1,630001,0
2,630002,0
3,630003,0
4,630004,0


In [19]:
submission = pd.DataFrame({
    "id":test_ids,
    "Heart Disease": test_preds 
})

In [20]:
submission.to_csv('submission.csv',index=False)